In [69]:
import numpy as np

In [83]:
# Dictionary with instruction op codes
mnemo_op_code = {
    "LDA": "0000",
    "ADD": "0001",
    "SUB": "0010",
    "OUT": "1110",
    "HLT": "1111"
}

def convert_mnemo_op_code(instruction):
    op_code = mnemo_op_code.get(instruction, "N/A")
    return op_code

# Function load
def load(file_path, ram_instance):

    # Open the file in reading
    with open(file_path, "r") as file:
        # Read each line
        for line_number, line in enumerate(file):
            # Delete blanks
            line = line.strip()

            # Split the line into instruction and hexadecimal number
            parts = line.split()
            instruction = parts[0]
            hex_number = parts[1]

            # Convert instruction to op code
            op_code = convert_mnemo_op_code(instruction)

            # Convert hexadecimal number to binary and take the last 4 digits
            binary_number = bin(int(hex_number, 16))[2:].zfill(4)[-4:]

            # Combine op code and binary number
            combined_data = op_code + binary_number

            # Append combined data to RAM using line number as address
            ram_instance.write(combined_data, line_number)

# Create an instance of the RAM class
my_ram = RAM()

# Call the load function with the file path and the RAM instance
load(r"file.txt", my_ram)

my_ram.read(1)


'00011011'

In [80]:
class Clock:
	def __init__():
		self.state=0
		self.on=True

	def advance(self, how_much):
		self.state+=how_much

	def __str__():
		return self.state

class RingCounter():
	def __init__(n_time_states):
		self.state=0
		self.n_time_states=n_time_states

	def advance(self, how_much):
		self.state=(self.state+how_much)%self.n_time_states

	def __str__():
		ring='0'*6
		ring[self.state]='1'
		return ring

In [ ]:
class Register:
    """
    Three 8-bit registers: A, B, IR. A and B are general-purpose, IR is the instruction register (store the current instruction that's being executed).
    Three numpy array of fixed size
    """

    def __init__(self):
        self.state = ''
    
    # Return the state of the register
    def read(self):
        return self.state
    
    # Overwrite the state of the register
    def write(self, data):
        self.state = data
    
    #TO DO 
    # print: similar to read but in a .txt convention
    def print(self):
        return self.state


class ByteRegister(Register):
    def __init__(self):
        self.state = '0'*8


class NibbleRegister(Register):
    def __init__(self):
        self.state = '0'*4
 
    
class RAM(Register):
    """
    RAM stores both the program and the data. 16 addresses of 8 bits each
    """

    def __init__(self):
        # List of 8-bits registers
        self.state = [ByteRegister() for _ in range(16)]

    # Write on the address of the RAM
    def write(self, data, address):
        # Data validation
        assert type(data) is str, 'data type error'
        assert len(data) == 8, 'size error'

        self.state[address].write(data)
    
    # Read the address of the RAM
    def read(self, address):
        return self.state[address].read()
    
    # TO DO
    # def print()


class InstructionRegister(ByteRegister):
    """
    Register where you can access the first and the second nibbles 
    """

    def readOpCode(self):
        return self.read()[:4]
    
    def readOperand(self):
        return self.read()[4:]


class ProgramCounter(NibbleRegister):
    """
    4-bits program counter
    """

    # Update the counter
    def advance(self):       
        # Get the decimal value
        value = int(self.state, 2)
        # Update the counter mod 2**4
        value = (value + 1)%2**4
        # Convert the new value into a 4-bits string
        string = bin(value)[2:].zfill(4)

        self.state = string

class Flag(Register):
    def __init__(self):
        self.state = 0
    

class ALU(ByteRegister):
    """
    The logic of the CPU
    """
    def __init__(self, aRegister, bRegister):
        self.a = aRegister
        self.b = bRegister
        
    #def read():


'1111'

In [36]:

#HELPER FUNCTIONS

#helper function for moving stuff between registers

def bus(reg1, reg2, cpu): #copy content of one register to another, it can  be a, b, ir, pc, mar, flag, output, alu, whatever
	cpu.advance_clock()
	reg2.write(reg1.read())


#MICRO-INSTRUCTIONS IMPLEMENTATION
	
#exemples of micro-instructions. Micro-instructions are functions of ONLY a cpu object:
	
def pc_to_mar(cpu):
	bus(cpu.pc,cpu.mar,cpu)

"""RAM.read() HA BISOGNO DI INDICE!"""
def ram_to_a(cpu):
	bus(cpu.ram, cpu.aRegister, cpu)

def ram_to_b(cpu):
	bus(cpu.ram, cpu.bRegister, cpu)

def ram_to_ir(cpu):
	cpu.instructionRegister.write(cpu.ram.read(cpu.memoryAddressRegister.read()))

def ir_to_mar(cpu):
	cpu.mar.write(cpu.readOperand())

def alu_to_a(cpu):
	bus(cpu.alu, cpu.aRegister, cpu)
	
def no_op(cpu):
	pass

def clock_off(cpu):
	cpu.clock.on=False

def fetch(cpu):
	pc_to_mar(cpu)
	ram_to_ir(cpu)
	cpu.programCounter.advance()

#dict of lists of functions, each list is addressed by an opcode
#corresponding to a macroinstruction available to the cpu,
#and contains functions that corresponds to micro-instructions.
#Very easy to expand the set, and very readable.

macro_instructions={ 
	'lda':[ir_to_mar, ram_to_a, no_op],
	'add':[ir_to_mar, ram_to_b, alu_to_a],
	'sub':[],
	'hlt':[clock_off, no_op, no_op],
}


In [37]:
class CPU:
	def __init__(): #inizializza a zero i clock, program counter e registri
		clock=Clock()
		ringCounter=RingCounter()
		programCounter=ProgramCounter()
		memoryAddressRegister=ByteRegister()
		aRegister=ByteRegister()
		bRegister=ByteRegister()
		instructionRegister=InstructionRegister()
		alu=ALU(aRegister,bRegister)
		flagA=Flag(a)	

	def load_ram(self, input_path):#parser, loads to ram the code
		pass
	def run(): #esegue il programma in ram
		clock.on=True 
		while(clock.on):
			fetch(self)
			advance_clock(3)
			while(clock.on):

				macro_instructions[mnemo_op_code[self.instructionRegister.readOpCode()]][self.ringCounter.read()](self)

                #questo esegue la micro-operazione dell'istruzione presente sull'ir,
                #corrispondente al time-state indicato sul microclock.

				advance_clock()
	
	def advance_clock(how_much=1):
		self.clock().advance(how_much)
		self.ringCounter().advance(how_much)

In [ ]:
cpu= CPU()
cpu.load_ram('mkmk')
cpu.run()


### SAVE

In [ ]:

#HELPER FUNCTIONS

#helper function for moving stuff between registers

def bus(reg1, reg2, cpu): #copy content of one register to another, it can  be a, b, ir, pc, mar, flag, output, alu, whatever
	cpu.advance_clock()
	reg2.write(reg1.read())


#MICRO-INSTRUCTIONS IMPLEMENTATION
	
#exemples of micro-instructions. Micro-instructions are functions of ONLY a cpu object:
	
def pc_to_mar(cpu):
	bus(cpu.pc,cpu.mar,cpu)

"""RAM.read() HA BISOGNO DI INDICE!"""
def ram_to_a(cpu):
	bus(cpu.ram, cpu.aRegister, cpu)

def ram_to_b(cpu):
	bus(cpu.ram, cpu.bRegister, cpu)

def ram_to_ir(cpu):
	cpu.instructionRegister.write(cpu.ram.read(cpu.memoryAddressRegister.read()))

def ir_to_mar(cpu):
	cpu.mar.write(cpu.readOperand())

def alu_to_a(cpu):
	bus(cpu.alu, cpu.aRegister, cpu)
	
def no_op(cpu):
	pass

def clock_off(cpu):
	cpu.clock.on=False

def fetch(cpu):
	pc_to_mar(cpu)
	ram_to_ir(cpu)
	cpu.programCounter.advance()

#dict of lists of functions, each list is addressed by an opcode
#corresponding to a macroinstruction available to the cpu,
#and contains functions that corresponds to micro-instructions.
#Very easy to expand the set, and very readable.

macro_instructions={ 
	'lda':[ir_to_mar, ram_to_a, no_op],
	'add':[ir_to_mar, ram_to_b, alu_to_a],
	'sub':[],
	'hlt':[clock_off, no_op, no_op],
}


In [ ]:
class CPU:
	def __init__(): #inizializza a zero i clock, program counter e registri
		clock=Clock()
		ringCounter=RingCounter()
		programCounter=ProgramCounter()
		memoryAddressRegister=ByteRegister()
		aRegister=ByteRegister()
		bRegister=ByteRegister()
		instructionRegister=InstructionRegister()
		alu=ALU(aRegister,bRegister)
		flagA=Flag(a)	

	def load_ram(self, input_path):#parser, loads to ram the code
		pass
	def run(): #esegue il programma in ram
		clock.on=True 
		while(clock.on):
			fetch(self)
			advance_clock(3)
			while(clock.on):

				macro_instructions[mnemo_op_code[self.instructionRegister.readOpCode()]][self.ringCounter.read()](self)

                #questo esegue la micro-operazione dell'istruzione presente sull'ir,
                #corrispondente al time-state indicato sul microclock.

				advance_clock()
	
	def advance_clock(how_much=1):
		self.clock().advance(how_much)
		self.ringCounter().advance(how_much)